In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
from BSDE import *
import warnings 
import bokeh
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_notebook, show, output_file
import time
warnings.filterwarnings("ignore")

In [6]:
T = 0.25
K = 100.
S0 = 100.
sigma = 0.2
p = 1
M = np.eye(p)
r = 0.01
mu = 0.05
R = 0.06
Q = 0.

In [31]:
M_run = 20
m = 12
N = 10000
n_picard = 3
n_estimators=100
learning_rate=0.1
max_depth=1
random_state=0
loss="ls"

In [18]:
a = np.zeros(M_run)
timing = np.zeros_like(a)
for i in range (M_run): 
    start = time.time()
    test_hd = BsdeHD(T, K, M, mu, Q, sigma, S0, r, R)
    a[i] = test_hd.get_price(N, m, option_type='call combination', option_payoff='geometric',
                             oType='European',n_picard=n_picard, regression='gbr')
    timing[i] = round(time.time() - start, 3)

In [21]:
a.mean(), timing.mean()

(2.9687040004205318, 9.5627000000000013)

In [22]:
import pandas as pd

In [25]:
param = ['q', 'mu', 'R ', 'r ','sigma', 'T', 'K', 'S0']
values = [Q, mu, R, r, sigma, T, K, S0]
dataset = list(zip(param,values))
df_param = pd.DataFrame(data = dataset, columns=['Parameters', 'values'])

In [35]:
sparam = ['N_particles', 'm_discretization', 'N_run', 'n_picard', 'Number of trees', 'Learning rate', 'max_depth']

svalues = [N, m, M_run, n_picard, n_estimators, learning_rate, max_depth, random_state]
sp_dataset = list(zip(sparam,svalues))
df_sparam = pd.DataFrame(sp_dataset, columns=['Specific Parameters', 'values'])
df_sparam

,Specific Parameters,values
0,N_particles,10000.0
1,m_discretization,12.0
2,N_run,20.0
3,n_picard,3.0
4,Number of trees,100.0
5,Learning rate,0.1
6,max_depth,1.0


In [36]:
run_id = ["run {}".format(i) for i in range(1, M_run+1)]
df = pd.DataFrame(list(zip(run_id, a, timing)), columns = ['run','values', 'time'])

In [37]:
min_a = round(df['values'].min(), 4)
max_a = round(df['values'].max(), 4)
mean_a = round(df['values'].mean(), 4)
std_a = round(df['values'].std(), 4)
left = round(mean_a - 1.96 * std_a / M_run, 4)
right = round(mean_a + 1.96 * std_a / M_run, 4)

In [38]:
stat_param = ['mean', 'std', '95% confidence interval', 'min', 'max']
stat_values = [mean_a, std_a, [left, right], min_a, max_a]
stat_data = list(zip(stat_param, stat_values))
stat_df = pd.DataFrame(stat_data, columns = ['stat parameter', 'values'])
stat_df

,stat parameter,values
0,mean,2.9687
1,std,0.0465
2,95% confidence interval,"[2.9641, 2.9733]"
3,min,2.886
4,max,3.0544


In [39]:
df_join = pd.concat([df_param, df_sparam, df, stat_df], axis = 0)
df_join.to_csv('output_GradBoost.csv')